<a href="https://colab.research.google.com/github/sonalvrshny/IR23-MRRS/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
%pip install nltk spacy tensorflow torch


In [2]:
%pip install langdetect

  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993253 sha256=96fef068856ce6231a54efb3661bef1508538f3ae67a74d8e8aa52877237de5c
  Stored in directory: c:\users\sonal\appdata\local\pip\cache\wheels\0a\f2\b2\e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [30]:
from langdetect import detect

def detect_language(query):
    try:
        return detect(query)
    except Exception as e:
        return "Error: " + str(e)
    
print(detect_language("how to make burger"))


en


In [38]:
# For english
!python -m spacy download en_core_web_sm

# For spanish
!python -m spacy download es_core_news_sm

# For hindi
!python -m spacy download xx_ent_wiki_sm


In [8]:
import spacy

# Load the language model for each language
nlp_en = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_sm")
nlp_hi = spacy.load("en_core_web_sm")  # Hindi is not directly supported, so using a multilingual model

def parse_query(query, lang):
    if lang == 'en':
        doc = nlp_en(query)
    elif lang == 'es':
        doc = nlp_es(query)
    elif lang == 'hi':
        doc = nlp_hi(query)
    else:
        return "Unsupported language"

    keywords = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return keywords


In [9]:
def process_query(query):
    lang = detect_language(query)
    if "Error" in lang:
        return lang
    keywords = parse_query(query, lang)
    return {"language": lang, "keywords": keywords}


In [15]:
# Load recipe JSON data into a list of dictionaries
import json
with open('recipes.json', 'r') as file:
    recipes = json.load(file)

print(recipes[0])

{'recipeName': 'मार्गेरीता पिज़्ज़ा', 'ingredients': ['2 कप मैदा (flour)', '1 कप पानी (water)', '1 छोटा चम्मच खमीर (1 tsp yeast)', '1 छोटा चम्मच नमक (1 tsp salt)', '1 बड़ा चम्मच जैतून का तेल (1 tbsp olive oil)', '1/2 कप टमाटर की सॉस (1/2 cup tomato sauce)', '1 कप मोज़ारेला चीज़ (1 cup mozzarella cheese)', 'ताजगी वाले तुलसी पत्तियां (Fresh basil leaves)', 'नमक और काली मिर्च स्वाद के अनुसार (Salt and pepper to taste)'], 'instruction': ['पानी में खमीर घोलकर मैदा, नमक, और तेल के साथ मिश्रित डो को बनाएं। 1 घंटे के लिए राइज़ होने दें।', 'ओवन को 475°F (245°C) पर प्रीहीट करें।', 'डो को बेलन से बेलें और टमाटर की सॉस लगाएं।', 'मोज़ारेला चीज़ और बेसिल पत्तियां डालें।', 'नमक और काली मिर्च से सीज़न करें।', 'क्रस्ट सुनहरा होने तक 10-15 मिनट तक बेक करें।']}


In [10]:
sample_queries = ["How to make traditional Mexican guacamole?", "Receta de paella de mariscos", "बटर चिकन की रेसिपी"]

for query in sample_queries:
    result = process_query(query)
    print(f"Query: {query}\nResult: {result}\n")


Query: How to make traditional Mexican guacamole?
Result: {'language': 'en', 'keywords': ['traditional', 'mexican', 'guacamole']}

Query: Receta de paella de mariscos
Result: {'language': 'es', 'keywords': ['Receta', 'paella', 'marisco']}

Query: बटर चिकन की रेसिपी
Result: {'language': 'hi', 'keywords': ['बटर']}



In [11]:
%pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/55.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/55.1 kB ? eta -:--:--
     --------------------- ---------------- 30.7/55.1 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 55.1/55.1 kB 477.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.5 MB 1.9 MB/s eta 0:00:01
     ----- ---------------------------------- 0.2/1.5 MB 2.1 MB/s eta 0:00:01
     ------- -------------------------------- 0.3/1.5 MB 2.4 MB/s eta 0:00:01
     ------------ --------------------------- 0.5/1.5 MB 2.6 MB/s eta 0:00:01
     ---------------- ----------------------- 0.6/1.5 MB 2.6 MB/s eta 0:00:01
     ---------------------- ----------------- 0.9/1.5 MB 3.1 MB/s eta 0:00:01
     ---------------------------- ----------- 1.1/1.5 

In [13]:
from googletrans import Translator

def translate_query(query, target_language):
    translator = Translator()
    translation = translator.translate(query, dest=target_language)
    return translation.text

def get_translated_queries(user_query):
    lang = detect_language(user_query)
    if lang == "hi":
        translated_queries = {
                'hindi': user_query,
                'spanish': translate_query(user_query, 'es'),
                'english': translate_query(user_query, 'en')
            }
    elif lang == "es":
        translated_queries = {
                'hindi': translate_query(user_query, 'hi'),
                'spanish': user_query,
                'english': translate_query(user_query, 'en')
            }
    elif lang == "en":
        translated_queries = {
                'hindi': translate_query(user_query, 'hi'),
                'spanish': translate_query(user_query, 'es'),
                'english': user_query
            }
    return translated_queries

user_queries = ["How to make traditional Mexican guacamole?", "Receta de paella de mariscos", "बटर चिकन की रेसिपी"]
for q in user_queries:
    print(get_translated_queries(q))

{'hindi': 'पारंपरिक मैक्सिकन गुआकामोल कैसे बनाएं?', 'spanish': '¿Cómo hacer guacamole mexicano tradicional?', 'english': 'How to make traditional Mexican guacamole?'}
{'hindi': 'सीफूड पेला नुस्खा', 'spanish': 'Receta de paella de mariscos', 'english': 'Seafood Paella Recipe'}
{'hindi': 'बटर चिकन की रेसिपी', 'spanish': 'Receta de pollo con mantequilla', 'english': 'Butter chicken recipe'}


In [21]:
import faiss
import numpy as np
def embed_text(text, model):
    return model.encode(text)

def build_index(database_embeddings):
    dim = len(database_embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(database_embeddings).astype('float32'))
    return index

def search_database(query_embedding, index, database, k=2):
    _, indices = index.search(np.array([query_embedding]).astype('float32'), k)
    # Return sorted list of recipes and their similarity scores
    return [{'recipe': database[i]['recipeName'], 'similarity': 1.0} for i in indices[0]]

In [36]:
from sentence_transformers import SentenceTransformer

# Load pre-trained model for sentence embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Extract recipe titles
recipe_titles = [item['recipeName'] for item in recipes]

# Embed all recipe texts
database_embeddings = [embed_text(text, model) for text in recipe_titles]

# Build FAISS index
index = build_index(database_embeddings)

user_query = "how to make chicken curry"
processed_query = process_query(user_query)
print(" ".join(processed_query['keywords']))
translated_queries = get_translated_queries(" ".join(processed_query['keywords']))
print(translated_queries)
# Embed the translated queries
query_embeddings = {lang: embed_text(translated_query, model) for lang, translated_query in translated_queries.items()}

# Search the database for each translated query using FAISS
results = {}
for lang, query_embedding in query_embeddings.items():
    results[lang] = search_database(query_embedding, index, recipes)

print(results)

chicken curry
{'hindi': 'चिकन करी', 'spanish': 'curry de pollo', 'english': 'chicken curry'}
{'hindi': [{'recipe': 'चिकन करी', 'similarity': 1.0}, {'recipe': 'पनीर टिक्का', 'similarity': 1.0}], 'spanish': [{'recipe': 'Pollo al Curry', 'similarity': 1.0}, {'recipe': 'Chicken Curry', 'similarity': 1.0}], 'english': [{'recipe': 'Chicken Curry', 'similarity': 1.0}, {'recipe': 'Pollo al Curry', 'similarity': 1.0}]}
